# Цель

Не менее 20000 символов.
уменьшить размерность текста, используя метод главных компонент, при этом обеспечить сохранение информации не менее 80%


# Задачи:

- осуществить загрузку текста, токенизацию

- почистить данные (числа, знаки препинания, символы, стоп-слова)

- осуществить лемматизацию

- перевести в векторное представление

- применить метод главных компонент таким образом, чтобы потеря информации составила не более 20%

- (если есть метки классов) посчитать результаты обучения. 


In [72]:
import pandas as pd
import spacy
import en_core_web_sm
import seaborn as sbn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import regex as re
from string import punctuation
from string import digits

## Подготовка модели

In [73]:
nlp = en_core_web_sm.load()

In [74]:
# читаем текст из файла и удаляем мусор
with open('text.txt') as f:
    text = f.read()

text = " ".join(re.sub(r'\[[0-9]+\]', '', text).split())

nlp.Defaults.stop_words.add("de")

doc = nlp(text)

## Лемматизация

Пропускаем цифры и знаки препинания

In [75]:
# лемматизация. пропускаем цифры и знаки препинания
sents = []
for sent in doc.sents:
    processed_sent = []
    for token in sent:
        if not token.is_stop:
            word = [char for char in token.lemma_.lower() if char not in digits and char not in punctuation]
            word = ''.join(word)
            if len(word) != 0:
                processed_sent.append(word)
    
    sents.append(' '.join(processed_sent))


sents = np.array(sents)

## Векторизация

In [76]:
vectorizer = TfidfVectorizer()

vectorized = vectorizer.fit_transform(sents)
vectorized

<702x2857 sparse matrix of type '<class 'numpy.float64'>'
	with 8912 stored elements in Compressed Sparse Row format>

In [77]:
df = pd.DataFrame(vectorized.toarray(), columns=vectorizer.get_feature_names_out())
df.shape

(702, 2857)

## Понижение размерности с помощью PCA

In [78]:
pca = PCA(390)
transformed = pca.fit_transform(df.T)

print(pca.explained_variance_ratio_.sum())
print(transformed.shape)

0.8008647408119611
(2857, 390)
